# Import libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Load processed dataset
The data we load here are already shuffled and normalized.

In [ ]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

# Pass input through autoencoders

In [ ]:
autoencoder_1_path = 'results/results-autoencoder-01-02/model/2018-03-05-18-34-05/weights.hdf5'
autoencoder_2_path = 'results/results-autoencoder-02-02/model/2018-03-05-20-49-19/weights.hdf5'
weights1 = load_model(autoencoder_1_path)
weights2 = load_model(autoencoder_2_path)

# Pass input through auto encoder #1
x_train_output = weights1.predict(x_train)
x_test_output = weights1.predict(x_test)

# Pass auto encoder #1 output through auto encoder #2
x_train_output = weights2.predict(x_train_output)
x_test_output = weights2.predict(x_test_output)

# Calculate accuracy

In [ ]:
classifier_path = 'results/results-classify-sae-04/model/2018-03-06-04-44-40/weights.hdf5'
classifier = load_model(classifier_path)
y_pred = classifier.predict(np.vstack([x_train_output, x_test_output]))
print(y_test.shape)
print(y_pred.shape)
acc = accuracy_score(np.vstack([y_train, y_test]), y_pred.round())
print(acc)


tn, fp, fn, tp = confusion_matrix(np.vstack([y_train.argmax(axis=1), y_test.argmax(axis=1)]), y_pred.round().argmax(axis=1)).ravel()

(tn, fp, fn, tp)
# print('accuracy by formula = ' + ((tp+tn)/(tp+fp+tn+fn)))